In [ ]:
# ===============================
# IMPORT & CONFIG
# ===============================
%matplotlib inline
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.dates as mdates
from IPython.display import display
import io

# ===============================
# CONFIGURAZIONE
# ===============================
# TICKER = "NFLX"  # es. AAPL # Rimosso per usare il valore dalla prima cella
# YEARS = 8        # ultimi anni da visualizzare # Rimosso per usare il valore dalla prima cella

# ===============================
# UTILITIES
# ===============================
BASE_URL = "https://discountingcashflows.com/company/{ticker}/{statement}/"

def get_table_discounting(ticker, statement):
    url = BASE_URL.format(ticker=ticker, statement=statement)
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        r = requests.get(url, headers=headers, timeout=20)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta per {statement}: {e}")
        return None
    soup = BeautifulSoup(r.text, "html.parser")
    tables = soup.find_all("table")
    if not tables:
        print(f"Nessuna tabella trovata per {statement} su {url}")
        return None
    for t in tables:
        try:
            df = pd.read_html(io.StringIO(str(t)))[0]
            if df.shape[1] >= 2:
                return df
        except:
            continue
    print(f"Nessuna tabella leggibile trovata per {statement} su {url}")
    return None

def extract_series_from_row(df, keywords):
    if df is None: return None, None
    periods = df.columns.tolist()[1:]
    for kw in keywords:
        mask = df.iloc[:,0].astype(str).str.contains(kw, case=False, na=False)
        if mask.any():
            row = df.loc[mask].iloc[0]
            vals = row.iloc[1:].astype(str).replace("-", np.nan).replace("", np.nan)
            cleaned_vals, cleaned_periods = [], []
            for i, v in enumerate(vals):
                if pd.isna(v): continue
                s = str(v).replace(",", "").replace("(", "-").replace(")", "").strip()
                try:
                    cleaned_vals.append(float(s))
                    cleaned_periods.append(periods[i])
                except:
                    continue
            if cleaned_vals:
                return cleaned_vals, cleaned_periods
    return None, None

# ===============================
# ESTRAZIONE FINANCIALS
# ===============================
def extract_financials(ticker):
    financials = {}
    # Income Statement
    is_df = get_table_discounting(ticker, "income-statement")
    financials["Revenue"], financials["Periods"] = extract_series_from_row(is_df, ["Revenue", "Sales", "Total Revenue"])
    financials["NetIncome"], _ = extract_series_from_row(is_df, ["Net Income", "Net loss", "NetLossProfit"])
    # Balance Sheet
    bs_df = get_table_discounting(ticker, "balance-sheet-statement")
    financials["Equity"], _ = extract_series_from_row(bs_df, ["Total Equity", "Total shareholders' equity", "Total stockholders' equity"])
    financials["Debt"], _ = extract_series_from_row(bs_df, ["Total Debt", "Total liabilities", "Total Long Term Debt"])
    # Cash Flow Statement
    cf_df = get_table_discounting(ticker, "cash-flow-statement")
    financials["FCF"], _ = extract_series_from_row(cf_df, ["Free Cash Flow", "Free cash flow", "FreeCashFlow", "Operating Cash Flow"])
    # Check if periods extracted
    if financials["Periods"] is None:
        print("Could not extract periods data. Cannot proceed with financial data processing.")
        return None
    # Limit last YEARS and invert order (vecchio → recente)
    for k in financials:
        if k != "Periods" and financials[k] is not None:
            financials[k] = financials[k][:YEARS][::-1]
    financials["Periods"] = financials["Periods"][:YEARS][::-1]
    return financials
# ===============================
# CALCOLO RATIOS
# ===============================
def calculate_ratios(financials):
    ratios = {}
    if financials is None: return None
    try:
        if financials["NetIncome"] is not None and financials["Revenue"] is not None and len(financials["NetIncome"]) == len(financials["Revenue"]):
            ratios["ProfitMargin"] = [ni/r*100 for ni,r in zip(financials["NetIncome"], financials["Revenue"])]
        else:
            ratios["ProfitMargin"] = None
    except:
        ratios["ProfitMargin"] = None
    try:
        if financials["NetIncome"] is not None and financials["Equity"] is not None and len(financials["NetIncome"]) == len(financials["Equity"]):
            ratios["ROE"] = [ni/e*100 for ni,e in zip(financials["NetIncome"], financials["Equity"])]
        else:
            ratios["ROE"] = None
    except:
        ratios["ROE"] = None
    try:
        if financials["Debt"] is not None and financials["Equity"] is not None and len(financials["Debt"]) == len(financials["Equity"]):
            ratios["DebtEquity"] = [d/e if e != 0 else np.nan for d,e in zip(financials["Debt"], financials["Equity"])]
        else:
            ratios["DebtEquity"] = None
    except:
        ratios["DebtEquity"] = None
    return ratios

# ===============================
# ESTRAZIONE STIME NET INCOME
# ===============================
def get_net_income_estimates(ticker):
    url = f"https://discountingcashflows.com/company/{ticker}/estimates/"
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        r = requests.get(url, headers=headers)
        r.raise_for_status()
    except:
        return None, None
    soup = BeautifulSoup(r.text, "html.parser")
    tables = soup.find_all("table")
    estimates_table = None
    for table in tables:
        if "Estimated Net Income" in table.text:
            estimates_table = table
            break
    if estimates_table is None: return None, None
    try:
        df = pd.read_html(io.StringIO(str(estimates_table)), header=None)[0]
    except:
        return None, None
    idx = df[df.iloc[:,0].astype(str).str.contains("Estimated Net Income", case=False, na=False)].index
    if idx.empty: return None, None
    low_row = df.iloc[idx[0]+1]
    numerical_cols_values = []
    for i in range(1, len(low_row)):
        val = low_row.iloc[i]
        try:
            s = str(val).replace(",", "").replace("(", "-").replace(")", "").strip()
            numerical_values = float(s)
            numerical_cols_values.append(numerical_values)
        except:
            continue
    current_year_estimate = numerical_cols_values[0] if len(numerical_cols_values)>=1 else None
    next_year_estimate = numerical_cols_values[1] if len(numerical_cols_values)>=2 else None
    return current_year_estimate, next_year_estimate

# ===============================
# PLOTTAGGIO FINANCIALS + RATIOS
# ===============================
def plot_financials_ratios_price(financials, ratios, ticker):
    if financials is None or ratios is None:
        print("Cannot plot due to missing financial or ratio data.")
        return

    labels = financials.get("Periods", [])
    rev = np.array(financials.get("Revenue", []), dtype=float) if financials.get("Revenue") is not None else np.array([])
    ni = np.array(financials.get("NetIncome", []), dtype=float) if financials.get("NetIncome") is not None else np.array([])

    # Adatta scala (Migliaia / Milioni)
    factor = 1
    max_val = 0
    if rev.size>0: max_val=max(max_val,max(rev))
    if ni.size>0: max_val=max(max_val,max(ni))
    if max_val>1e6: factor=1e6
    if rev.size>0: rev/=factor
    if ni.size>0: ni/=factor
    y_label_fin = "Amount"
    if factor==1e6: y_label_fin+=" (M)"

    # Ratios
    pm = np.array(ratios.get("ProfitMargin", np.zeros(len(labels))))
    roe = np.array(ratios.get("ROE", np.zeros(len(labels))))
    de = np.array(ratios.get("DebtEquity", np.zeros(len(labels))))

    # ===============================
    # PLOT FINANCIALS + RATIOS
    # ===============================
    fig, axs = plt.subplots(2,1, figsize=(14,10))
    # --- Financials
    if len(labels)==len(rev) and len(labels)==len(ni):
        axs[0].plot(labels, rev, marker='o', label="Revenue")
        axs[0].plot(labels, ni, marker='o', label="Net Income")
        axs[0].set_title(f"{ticker} Financials")
        axs[0].set_ylabel(y_label_fin)
        axs[0].legend()
        axs[0].grid(True)
    else:
        axs[0].set_title(f"{ticker} Financials (Data Unavailable)")
        axs[0].text(0.5,0.5,"Financial data not available.", ha='center', va='center', transform=axs[0].transAxes)

    # --- Ratios
    if len(labels)==len(pm) and len(labels)==len(roe) and len(labels)==len(de):
        axs[1].plot(labels, pm, marker='o', label="Profit Margin %")
        axs[1].plot(labels, roe, marker='o', label="ROE %")
        axs[1].plot(labels, de, marker='o', label="Debt/Equity")
        axs[1].set_title(f"{ticker} Ratios")
        axs[1].set_ylabel("Ratio / %")
        axs[1].legend()
        axs[1].grid(True)

        # Add labels to the ratio plot
        for i, label in enumerate(labels):
            if not np.isnan(pm[i]):
                axs[1].text(label, pm[i], f'{pm[i]:.1f}%', ha='left', va='bottom')
            if not np.isnan(roe[i]):
                axs[1].text(label, roe[i], f'{roe[i]:.1f}%', ha='left', va='bottom')
            if not np.isnan(de[i]):
                 axs[1].text(label, de[i], f'{de[i]:.2f}', ha='left', va='bottom') # Debt/Equity as a ratio, not percentage

    else:
        axs[1].set_title(f"{ticker} Ratios (Data Unavailable)")
        axs[1].text(0.5,0.5,"Ratio data not available.", ha='center', va='center', transform=axs[1].transAxes)


    plt.tight_layout()
    plt.show()


# Added calls to get financial data and ratio data first
financial_data = extract_financials(TICKER)
ratio_data = calculate_ratios(financial_data)
plot_financials_ratios_price(financial_data, ratio_data, TICKER)


# ===============================
# ===============================
# TERZO BLOCCO: NET INCOME + STOCK PRICE (assi X/Y corretti)
# ===============================

# Creazione DataFrame Net Income storico + stime
historical_ni = financial_data.get("NetIncome", [])
historical_periods = financial_data.get("Periods", [])

historical_data_with_years = []
if historical_ni and historical_periods and len(historical_ni) == len(historical_periods):
    for period, ni in zip(historical_periods, historical_ni):
        try:
            year = int(pd.to_datetime(period).year)
            historical_data_with_years.append({'Year': year, 'Net Income': ni, 'Type': 'Historical'})
        except:
            continue

# Estrazione stime Net Income
estimate_for_2025 = None
estimate_for_2026 = None

url = f"https://discountingcashflows.com/company/{TICKER}/estimates/"
headers = {"User-Agent": "Mozilla/5.0"}
try:
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")
    tables = soup.find_all("table")

    estimates_table = None
    for table in tables:
        if "Estimated Net Income" in table.text:
            estimates_table = table
            break

    if estimates_table:
        df_estimates = pd.read_html(io.StringIO(str(estimates_table)), header=None)[0]
        estimated_net_income_row_index = df_estimates[df_estimates.iloc[:, 0].astype(str).str.contains("Estimated Net Income", case=False, na=False)].index

        if not estimated_net_income_row_index.empty:
            low_row_index = estimated_net_income_row_index[0] + 1
            low_row = df_estimates.iloc[low_row_index]

            numerical_cols_values = []
            for i in range(1, len(low_row)):
                val = low_row.iloc[i]
                try:
                    s = str(val).replace(",", "").replace("(", "-").replace(")", "").strip()
                    numerical_values = float(s)
                    numerical_cols_values.append(numerical_values)
                except:
                    pass

            # 5° valore → stima 2025, 4° valore → stima 2026
            if len(numerical_cols_values) > 4: estimate_for_2025 = numerical_cols_values[4]
            if len(numerical_cols_values) > 3: estimate_for_2026 = numerical_cols_values[3]

except:
    pass

# Aggiungi le stime
estimated_data = []
if estimate_for_2025 is not None: estimated_data.append({'Year': 2025, 'Net Income': estimate_for_2025, 'Type': 'Estimated'})
if estimate_for_2026 is not None: estimated_data.append({'Year': 2026, 'Net Income': estimate_for_2026, 'Type': 'Estimated'})

# Combina storico + stimato
ni_df = pd.DataFrame(historical_data_with_years + estimated_data).sort_values(by='Year').reset_index(drop=True)

# ===============================
# Stock Price
# ===============================
stock_data = pd.DataFrame()  # Initialize stock_data as an empty DataFrame
price_col = None # Initialize price_col to None

try:
    start_date_stock = (datetime.now() - timedelta(days=YEARS * 365 * 1.1)).strftime('%Y-%m-%d') # Fetch slightly more than YEARS to ensure coverage
    end_date_stock = datetime.now().strftime('%Y-%m-%d')
    stock_data = yf.download(TICKER, start=start_date_stock, end=end_date_stock, progress=False, auto_adjust=False) # Added auto_adjust=False

    if not stock_data.empty:
        # Scegli la colonna prezzo
        if 'Adj Close' in stock_data.columns:
            price_col = 'Adj Close'
        elif 'Close' in stock_data.columns:
            price_col = 'Close'
        else:
            print("Nessuna colonna Close/Adj Close trovata nei dati delle azioni.")
            stock_data = pd.DataFrame() # Imposta stock_data a un DataFrame vuoto se non ci sono colonne di prezzo valide
            price_col = None

        # Filter stock_data to include only the last YEARS if stock_data is not empty
        if not stock_data.empty:
            start_date_filter = datetime.now() - timedelta(days=YEARS * 365)
            stock_data = stock_data[stock_data.index >= start_date_filter]


except Exception as e:
    print("Errore fetching stock data:", e)
    # If fetching stock data fails, stock_data and price_col remain as initialized empty DataFrame and None


# ===============================
# PLOT
# ===============================
fig, ax3 = plt.subplots(figsize=(14,6))

# Linea arancione: Historical Net Income
hist_ni = ni_df[ni_df['Type'] == 'Historical']
ax3.plot(pd.to_datetime(hist_ni['Year'].astype(str) + "-12-31"),
         hist_ni['Net Income'], marker='o', color='orange', label='Historical Net Income')

# Linea rossa tratteggiata: Estimated Net Income
est_ni = ni_df[ni_df['Type'] == 'Estimated']
ax3.plot(pd.to_datetime(est_ni['Year'].astype(str) + "-12-31"),
         est_ni['Net Income'], marker='o', color='red', linestyle='--', label='Estimated Net Income')

# Linea di continuità tra ultimo storico e prima stima
if not hist_ni.empty and not est_ni.empty:
    last_hist_date = pd.to_datetime(hist_ni['Year'].iloc[-1].astype(str) + "-12-31")
    last_hist_value = hist_ni['Net Income'].iloc[-1]
    first_est_date = pd.to_datetime(est_ni['Year'].iloc[0].astype(str) + "-12-31")
    first_est_value = est_ni['Net Income'].iloc[0]
    ax3.plot([last_hist_date, first_est_date], [last_hist_value, first_est_value],
             color='red', linestyle='--', alpha=0.7)

ax3.set_xlabel("Year")
ax3.set_ylabel("Net Income")
ax3.set_title(f"{TICKER} Net Income + Stock Price")
ax3.grid(True)
ax3.legend(loc='upper left')

# ===============================
# Stock Price su asse Y destro (stesso X di Net Income)
# ===============================
if 'stock_data' in globals() and not stock_data.empty and price_col in stock_data.columns:
    ax4 = ax3.twinx()
    ax4.plot(stock_data.index, stock_data[price_col], color='blue', alpha=0.5, label='Stock Price')
    ax4.set_ylabel("Stock Price")
    ax4.legend(loc='upper right')
    # Allineiamo X
    min_date = min(pd.to_datetime(hist_ni['Year'].iloc[0].astype(str) + "-12-31"), stock_data.index.min())
    max_date = max(pd.to_datetime(est_ni['Year'].iloc[-1].astype(str) + "-12-31"), stock_data.index.max())
    ax3.set_xlim(min_date, max_date)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# IMPORT & CONFIG
# ===============================
%matplotlib inline
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from IPython.display import display
import io
import yfinance as yf
from datetime import datetime

# ===============================
# CONFIGURAZIONE
# ===============================
TICKER = "STMMI.MI"  # es. AAPL
YEARS = 4        # ultimi anni da visualizzare

# ===============================
# BASE URL
# ===============================
BASE_URL = "https://discountingcashflows.com/company/{ticker}/{statement}/"

# ===============================
# FUNZIONI DI BASE
# ===============================
def get_table_discounting(ticker, statement):
    url = BASE_URL.format(ticker=ticker, statement=statement)
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        r = requests.get(url, headers=headers, timeout=20)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta per {statement}: {e}")
        return None

    soup = BeautifulSoup(r.text, "html.parser")
    tables = soup.find_all("table")
    if not tables:
        print(f"Nessuna tabella trovata per {statement} su {url}")
        return None
    for t in tables:
        try:
            df = pd.read_html(io.StringIO(str(t)))[0]
            if df.shape[1] >= 2:
                return df
        except:
            continue
    return None


def extract_value(df, keywords):
    if df is None:
        return 0
    for keyword in keywords:
        mask = df.iloc[:, 0].astype(str).str.contains(keyword, case=False, na=False)
        if mask.any():
            row = df[mask].iloc[0, 1:].dropna()
            if len(row) == 0:
                continue
            val = row.iloc[0]
            if isinstance(val, str):
                # Remove commas, parentheses, and handle negative numbers in parentheses
                val = float(re.sub(r'[(),]', '', val.replace(',', '').replace('(', '-').replace(')', '')))
            return float(val)
    return 0


def extract_row(df, keywords):
    if df is None:
        return None, None
    periods = df.columns.tolist()[1:]
    for kw in keywords:
        mask = df.iloc[:, 0].astype(str).str.contains(kw, case=False, na=False)
        if mask.any():
            row = df.loc[mask].iloc[0]
            vals = row.iloc[1:].astype(str).replace("-", np.nan).replace("", np.nan)
            cleaned_vals, cleaned_periods = [], []
            for i, v in enumerate(vals):
                if pd.isna(v):
                    cleaned_vals.append(np.nan)
                    cleaned_periods.append(periods[i])
                else:
                    try:
                        s = str(v).replace(",", "").replace("(", "-").replace(")", "").strip()
                        cleaned_vals.append(float(s))
                        cleaned_periods.append(periods[i])
                    except:
                        cleaned_vals.append(np.nan)
                        cleaned_periods.append(periods[i])
            return cleaned_vals, cleaned_periods
    return None, None


def extract_market_cap_and_shares_from_overview(ticker):
    """
    Estrae Market Cap e Shares Outstanding dalla pagina overview.
    Ritorna i valori numerici e gli indicatori di scala.
    """
    url = BASE_URL.format(ticker=ticker, statement="overview")
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        r = requests.get(url, headers=headers, timeout=20)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta per overview: {e}")
        return None, None, None, None

    soup = BeautifulSoup(r.text, "html.parser")

    market_cap_value, market_cap_scale_indicator = None, None
    shares_outstanding_value, shares_outstanding_scale_indicator = None, None

    # Estrai Market Cap
    market_cap_text_element = soup.find(string=lambda text: text and "Market Cap" in text)
    if market_cap_text_element:
        value_element = market_cap_text_element.find_next()
        if value_element:
            raw_value_text = value_element.text.strip()
            match = re.search(r'([\d,]+\.?\d*)\s*(Bil|Mil|Thou|billion|million|thousand)?', raw_value_text, re.IGNORECASE)
            if match:
                value_str = match.group(1).replace(',', '')
                market_cap_scale_indicator = match.group(2)
                try:
                    market_cap_value = float(value_str)
                except ValueError:
                    pass # Keep as None

    # Estrai Shares Outstanding - cerca vicino a Market Cap o come voce separata
    shares_outstanding_text_element = soup.find(string=lambda text: text and ("Shares Outstanding" in text or "Shares" in text))
    if shares_outstanding_text_element:
         value_element = shares_outstanding_text_element.find_next()
         if value_element:
            raw_value_text = value_element.text.strip()
            match = re.search(r'([\d,]+\.?\d*)\s*(Bil|Mil|Thou|billion|million|thousand)?', raw_value_text, re.IGNORECASE)
            if match:
                value_str = match.group(1).replace(',', '')
                shares_outstanding_scale_indicator = match.group(2)
                try:
                    shares_outstanding_value = float(value_str)
                except ValueError:
                    pass # Keep as None


    return market_cap_value, market_cap_scale_indicator, shares_outstanding_value, shares_outstanding_scale_indicator


def get_market_cap_and_shares_values(ticker):
    """
    Combina estrazione Market Cap e Shares Outstanding dall'overview.
    Ritorna i valori numerici e i loro indicatori di scala.
    """
    market_cap_value, market_cap_scale_indicator, shares_outstanding_value, shares_outstanding_scale_indicator = extract_market_cap_and_shares_from_overview(ticker)


    # Fallback to income statement if shares not found in overview
    if shares_outstanding_value is None or np.isnan(shares_outstanding_value):
         is_df = get_table_discounting(ticker, "income-statement")
         so_full, so_periods_full = extract_row(is_df, [
            "Diluted Weighted Average Shares Outstanding",
            "Weighted Average Shares Outstanding",
            "Shares Outstanding",
            "Common Stock Shares Outstanding",
            "Weighted Average Shares",
            "Common Stock Shares"
         ])
         shares_outstanding_value = so_full[0] if so_full and len(so_full) > 0 else np.nan
         shares_outstanding_scale_indicator = None # Reset scale indicator if using fallback


    return market_cap_value, market_cap_scale_indicator, shares_outstanding_value, shares_outstanding_scale_indicator

def get_last_two_periods_data(values, periods):
    """
    Given a list of values and periods (most recent first),
    returns the value and period for the last two available periods,
    including LTM if present.
    """
    if values is None or periods is None or len(values) < 2 or len(values) != len(periods):
        return None, None, None, None # Need at least 2 values and matching periods

    # values and periods are already sorted most recent first from extract_row
    latest_value, latest_period = values[0], periods[0]
    previous_value, previous_period = values[1], periods[1]

    return latest_value, latest_period, previous_value, previous_period


# ===============================
# CALCOLO YIELD
# ===============================
def calculate_shareholder_yield(ticker):
    print(f"--- Estrazione dati per {ticker} ---")
    cf_df = get_table_discounting(ticker, "cash-flow-statement")

    if cf_df is None:
        print(f"❌ Cash Flow non trovato per {ticker}")
        return None


    # Estrai Market Cap e Shares Outstanding dal sito
    market_cap_value, market_cap_scale_indicator, shares_outstanding_value, shares_outstanding_scale_indicator = get_market_cap_and_shares_values(ticker)


    # Prezzo attuale e EPS/PE da Yahoo Finance
    current_price = None
    eps_value = None
    pe_ratio_value = None

    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        current_price = info.get('regularMarketPrice')
        eps_value = info.get('trailingEps') # Or forwardEps depending on preference
        pe_ratio_value = info.get('trailingPE') # Or forwardPE depending on preference

    except Exception as e:
        print(f"Errore durante l'estrazione da Yahoo Finance: {e}")


    # Estrazione valori Cash Flow (dal sito originale)
    dividends = abs(extract_value(cf_df, ["Dividends Paid"]))
    buyback = abs(extract_value(cf_df, ["Repurchase", "Common Stock Repurchased"]))
    issuance = abs(extract_value(cf_df, ["Issuance of Stock", "Common Stock Issued"]))
    # Keep Debt Repayment extraction for the original Shareholder Yield calculation - EXTRACT WITHOUT ABS
    debt_repayment_cf = extract_value(cf_df, ["Debt Repayment"]) # Extract without absolute value


    # Calcolo per azione (usando i valori estratti direttamente, assumendo scala coerente)
    shareholder_y = None
    dividend_y = None
    buyback_y = None


    if current_price is not None and isinstance(current_price, (int, float)) and shares_outstanding_value is not None and not np.isnan(shares_outstanding_value) and shares_outstanding_value != 0 and market_cap_value is not None:

        # Usiamo i values extracted directly, assuming they are in the same scale
        # The values extracted from the website are likely in millions or billions,
        # while shares outstanding is also in millions or billions.
        # The per share calculation needs to account for the scale difference.
        # Let's use the Market Cap as the base for the yield calculation,
        # as it is directly comparable to the financial statement values (assuming they are in the same scale).
        # We need to convert the market cap to the same scale as the financial statement values.

        # Determine the scale factor based on the financial data extracted (Dividends, Buyback, Issuance, Debt Repayment)
        # Assume the scale of these values is consistent. We need to infer it.
        # A simple heuristic: check the magnitude of the values relative to the reported scale indicator of Shares Outstanding or Market Cap.
        # A more robust approach would be to look for scale indicators on the financial statement tables, but the function `get_table_discounting` doesn't extract this.
        # For now, let's assume the financial statement values (Dividends, Buyback, Issuance, Debt Repayment) are in the same scale as indicated by the Shares Outstanding.

        financial_data_scale_factor = 1.0
        if shares_outstanding_scale_indicator:
             if shares_outstanding_scale_indicator.lower() in ['bil', 'billion']:
                 financial_data_scale_factor = 1e9
             elif shares_outstanding_scale_indicator.lower() in ['mil', 'million']:
                 financial_data_scale_factor = 1e6
             elif shares_outstanding_scale_indicator.lower() in ['thou', 'thousand']:
                 financial_data_scale_factor = 1e3
             else:
                  financial_data_scale_factor = 1.0
        # If no shares outstanding scale indicator, try market cap scale indicator
        elif market_cap_scale_indicator:
             if market_cap_scale_indicator.lower() in ['bil', 'billion']:
                 financial_data_scale_factor = 1e9
             elif market_cap_scale_indicator.lower() in ['mil', 'million']:
                 financial_data_scale_factor = 1e6
             elif market_cap_scale_indicator.lower() in ['thou', 'thousand']:
                 financial_data_scale_factor = 1e3
             else:
                  financial_data_scale_factor = 1.0
        else:
             # If no scale indicator found, assume the values are in the base unit (e.g., USD)
             financial_data_scale_factor = 1.0


        # Scale the Market Cap to match the financial data scale
        market_cap_scaled = np.nan
        if market_cap_value is not None and market_cap_scale_indicator:
            mc_scale_factor = 1.0
            if market_cap_scale_indicator.lower() in ['bil', 'billion']:
                 mc_scale_factor = 1e9
            elif market_cap_scale_indicator.lower() in ['mil', 'million']:
                 mc_scale_factor = 1e6
            elif market_cap_scale_indicator.lower() in ['thou', 'thousand']:
                 mc_scale_factor = 1e3

            if financial_data_scale_factor != 0:
                 market_cap_scaled = market_cap_value * (mc_scale_factor / financial_data_scale_factor)
            else:
                market_cap_scaled = np.nan
        # If market cap value is present but no indicator, assume it's in the same scale as inferred financial data
        elif market_cap_value is not None:
             market_cap_scaled = market_cap_value # Assume same scale if no indicator

        else:
            market_cap_scaled = np.nan


        if market_cap_scaled is not None and not np.isnan(market_cap_scaled) and market_cap_scaled != 0:
             dividends_scaled = dividends if not np.isnan(dividends) else 0
             buyback_scaled = buyback if not np.isnan(buyback) else 0
             issuance_scaled = issuance if not np.isnan(issuance) else 0
             # Use the Debt Repayment from CF, apply the correct sign logic for the original calculation
             debt_component_scaled = -debt_repayment_cf if not np.isnan(debt_repayment_cf) else 0 # If positive (issuance), subtract; if negative (repayment), add (by subtracting the negative)


             # Recalculate Shareholder Yield based on scaled values and Market Cap
             shareholder_y_original = ((dividends_scaled + (buyback_scaled - issuance_scaled) + debt_component_scaled) / market_cap_scaled) * 100


        else:
            print("⚠️ Market Cap Scaled is not available or is zero. Cannot calculate Shareholder Yield.")
            shareholder_y_original = np.nan


    else:
        print("⚠️ Dati insufficienti per calcolare Yields (Prezzo attuale, Shares Outstanding, o Market Cap mancanti).")
        shareholder_y_original = np.nan


    # Formatta Market Cap per la visualizzazione
    formatted_market_cap = "N/A"
    if market_cap_value is not None:
        if market_cap_scale_indicator:
             if market_cap_scale_indicator.lower() in ['bil', 'billion']:
                 formatted_market_cap = f"{round(market_cap_value, 2)} Bil"
             elif market_cap_scale_indicator.lower() in ['mil', 'million']:
                 formatted_market_cap = f"{round(market_cap_value, 2)} Mil"
             elif market_cap_scale_indicator.lower() in ['thou', 'thousand']:
                 formatted_market_cap = f"{round(market_cap_value, 2)} Thou"
             else:
                 # Fallback se l'indicatore non è standard ma il valore è presente
                 if market_cap_value >= 1e9:
                     formatted_market_cap = f"{round(market_cap_value / 1e9, 2)} Bil (stima)"
                 elif market_cap_value >= 1e6:
                     formatted_market_cap = f"{round(market_cap_value / 1e6, 2)} Mil (stima)"
                 elif market_cap_value >= 1e3:
                     formatted_market_cap = f"{round(market_cap_value / 1e3, 2)} Thou (stima)"
                 else:
                     formatted_market_cap = f"{market_cap_value:,.0f}"
        else:
             # Tentativo di formattare anche senza indicatore esplicito dall'overview
             if market_cap_value is not None:
                 if market_cap_value >= 1e9:
                     formatted_market_cap = f"{round(market_cap_value / 1e9, 2)} Bil (stima)"
                 elif market_cap_value >= 1e6:
                     formatted_market_cap = f"{round(market_cap_value / 1e6, 2)} Mil (stima)"
                 elif market_cap_value >= 1e3:
                     formatted_market_cap = f"{round(market_cap_value / 1e3, 2)} Thou (stima)"
                 else:
                     formatted_market_cap = f"{market_cap_value:,.0f}"


    # Formatta Shares Outstanding per la visualizzazione
    formatted_shares_outstanding = "N/A"
    if shares_outstanding_value is not None and not np.isnan(shares_outstanding_value):
        if shares_outstanding_scale_indicator:
             if shares_outstanding_scale_indicator.lower() in ['bil', 'billion']:
                 formatted_shares_outstanding = f"{round(shares_outstanding_value, 2)} Bil"
             elif shares_outstanding_scale_indicator.lower() in ['mil', 'million']:
                 formatted_shares_outstanding = f"{round(shares_outstanding_value, 2)} Mil"
             elif shares_outstanding_scale_indicator.lower() in ['thou', 'thousand']:
                 formatted_shares_outstanding = f"{round(shares_outstanding_value, 2)} Thou"
             else:
                 # Fallback se l'indicatore non è standard ma il valore è presente
                 if shares_outstanding_value >= 1e9:
                     formatted_shares_outstanding = f"{round(shares_outstanding_value / 1e9, 2)} Bil (stima)"
                 elif shares_outstanding_value >= 1e6:
                     formatted_shares_outstanding = f"{round(shares_outstanding_value / 1e6, 2)} Mil (stima)"
                 elif shares_outstanding_value >= 1e3:
                     formatted_shares_outstanding = f"{round(shares_outstanding_value / 1e3, 2)} Thou (stima)"
                 else:
                     formatted_shares_outstanding = f"{shares_outstanding_value:,.0f}"

        else:
             # Fallback to heuristic if no indicator is extracted
             if shares_outstanding_value is not None and not np.isnan(shares_outstanding_value):
                 if shares_outstanding_value >= 1e9:
                     formatted_shares_outstanding = f"{round(shares_outstanding_value / 1e9, 2)} Bil (stima)"
                 elif shares_outstanding_value >= 1e6:
                     formatted_shares_outstanding = f"{round(shares_outstanding_value / 1e6, 2)} Mil (stima)"
                 elif shares_outstanding_value >= 1e3:
                     formatted_shares_outstanding = f"{round(shares_outstanding_value / 1e3, 2)} Thou (stima)"
                 else:
                     formatted_shares_outstanding = f"{shares_outstanding_value:,.0f}"


    return {
        "Ticker": ticker,
        "Current Price": round(current_price, 2) if isinstance(current_price, (int, float)) else "N/A",
        "Shares Outstanding": formatted_shares_outstanding, # Usa il valore formattato
        "Market Cap": formatted_market_cap, # Usa il valore formattato
        "Dividends": dividends if not np.isnan(dividends) else "N/A", # Use extracted values, handle NaN
        "Buyback": buyback if not np.isnan(buyback) else "N/A", # Use extracted values, handle NaN
        "Issuance": issuance if not np.isnan(issuance) else "N/A", # Use extracted values, handle NaN
        "Debt Repayment (CF)": debt_repayment_cf if not np.isnan(debt_repayment_cf) else "N/A", # Keep Debt Repayment from CF (raw value)
        "Shareholder Yield (%)": round(shareholder_y_original, 2) if shareholder_y_original is not None and not np.isnan(shareholder_y_original) else "N/A", # Ridenominato per chiarezza
        "EPS": round(eps_value, 2) if isinstance(eps_value, (int, float)) else "N/A", # Aggiunto EPS da Yahoo Finance
        "PE Ratio": round(pe_ratio_value, 2) if isinstance(pe_ratio_value, (int, float)) else "N/A" # Aggiunto PE Ratio da Yahoo Finance
    }


# ===============================
# ESECUZIONE
# ===============================
result = calculate_shareholder_yield(TICKER)

# Mostra EPS e PE prima della tabella principale
# Modificato per mostrare N/A solo se i valori non sono stati trovati
if result.get('EPS') == 'N/A':
    print("⚠️ EPS non trovato (finale)")
else:
    print(f"EPS: {result.get('EPS')} - Fonte: Yahoo Finance")

if result.get('PE Ratio') == 'N/A':
    print("⚠️ PE Ratio non trovato (finale)")
else:
    print(f"PE Ratio: {result.get('PE Ratio')} - Fonte: Yahoo Finance")

print("-" * 30) # Separatore per chiarezza

# Prepare data for the main table, including the new components and the calculated shareholder yield
result_for_table = {
    "Ticker": result.get("Ticker"),
    "Current Price": result.get("Current Price"),
    "Shares Outstanding": result.get("Shares Outstanding"),
    "Market Cap": result.get("Market Cap"),
    "Dividends": result.get("Dividends"),
    "Buyback": result.get("Buyback"),
    "Issuance": result.get("Issuance"),
    "Debt Repayment (CF)": result.get("Debt Repayment (CF)"), # Keep Debt Repayment from CF
    "Shareholder Yield (%)": result.get("Shareholder Yield (%)") # Ridenominato
}


display(pd.DataFrame([result_for_table])) # Commented out to remove table display

In [ ]:
# ==============================================
# 📊 ANALISI BUYBACK AZIONI - COMPLETO
# ==============================================

%matplotlib inline
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import io

# ===============================
# CONFIGURAZIONE
# ===============================
# Usa i valori definiti nella prima cella
# TICKER = "AAPL"   # cambia ticker # Rimosso per usare il valore dalla prima cella
# YEARS = 8         # numero di anni da visualizzare # Rimosso per usare il valore dalla prima cella
DILUTED = False   # False → Weighted Average Shares, True → Diluted Weighted Average Shares

# ===============================
# FUNZIONI BASE
# ===============================
BASE_URL = "https://discountingcashflows.com/company/{ticker}/{statement}/"

def get_table_discounting(ticker, statement):
    """Scarica e restituisce la tabella HTML di uno statement da discountingcashflows.com"""
    url = BASE_URL.format(ticker=ticker, statement=statement)
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        r = requests.get(url, headers=headers, timeout=20)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta per {statement}: {e}")
        return None

    soup = BeautifulSoup(r.text, "html.parser")
    tables = soup.find_all("table")
    if not tables:
        print(f"Nessuna tabella trovata per {statement} su {url}")
        return None

    for t in tables:
        try:
            df = pd.read_html(io.StringIO(str(t)))[0]
            if df.shape[1] >= 2:
                return df
        except:
            continue

    print(f"Nessuna tabella leggibile trovata per {statement} su {url}")
    return None


def extract_series_from_row(df, keywords):
    """Cerca nel dataframe una riga contenente una delle keyword e restituisce valori e periodi."""
    if df is None:
        return None, None
    periods = df.columns.tolist()[1:]
    for kw in keywords:
        mask = df.iloc[:, 0].astype(str).str.contains(kw, case=False, na=False)
        if mask.any():
            row = df.loc[mask].iloc[0]
            vals = row.iloc[1:].astype(str).replace("-", np.nan).replace("", np.nan)
            cleaned_vals, cleaned_periods = [], []
            for i, v in enumerate(vals):
                if pd.isna(v):
                    continue
                s = str(v).replace(",", "").replace("(", "-").replace(")", "").strip()
                try:
                    cleaned_vals.append(float(s))
                    cleaned_periods.append(periods[i])
                except:
                    continue
            if cleaned_vals:
                return cleaned_vals, cleaned_periods
    return None, None


# ===============================
# ANALISI BUYBACK
# ===============================
def analyze_share_count_trend_income(ticker, years=8, diluted=False):
    # Estrae l'Income Statement
    is_df = get_table_discounting(ticker, "income-statement")

    # Keyword da cercare
    if diluted:
        keywords = ["Diluted Weighted Average Shares Outstanding"]
    else:
        keywords = ["Weighted Average Shares Outstanding"]

    shares, periods = extract_series_from_row(is_df, keywords)

    if shares is None or len(shares) < 2:
        print(f"📉 Non disponibili dati sufficienti per analizzare le azioni in circolazione di {ticker}")
        return None

    # Normalizza (vecchio → recente)
    shares = shares[::-1]
    periods = periods[::-1]

    # Limita agli ultimi years
    shares = shares[-years:]
    periods = periods[-years:]

    # Calcola variazione percentuale
    variation = (shares[-1] - shares[0]) / shares[0] * 100

    print(f"\n📊 Analisi numero azioni in circolazione per {ticker}")
    print(f"Periodo analizzato: {periods[0]} → {periods[-1]}")
    print(f"Azioni (inizio → fine): {shares[0]:,.0f} → {shares[-1]:,.0f}")
    print(f"Variazione: {variation:.2f}%")

    # Interpretazione
    if variation < -3:
        print("🟢 Possibile BUYBACK (riduzione significativa delle azioni in circolazione)")
    elif variation > 3:
        print("🔴 Possibile DILUIZIONE (aumento delle azioni in circolazione)")
    else:
        print("🟡 Numero di azioni stabile negli ultimi anni")

    # ===============================
    # GRAFICO
    # ===============================
    plt.figure(figsize=(8,4))
    plt.plot(periods, shares, marker="o", linewidth=2)
    plt.title(f"📈 Andamento azioni in circolazione - {ticker}")
    plt.xlabel("Anno")
    plt.ylabel("Azioni (unità)")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return {"Periods": periods, "Shares": shares, "Variation(%)": variation}


# ===============================
# ESECUZIONE
# ===============================
result = analyze_share_count_trend_income(TICKER, YEARS, DILUTED)

In [ ]:
# ==========================================
# IMPORT & CONFIG
# ==========================================
%matplotlib inline
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import io

# ==========================================
# CONFIGURAZIONE
# ==========================================
# Usa i valori definiti nella prima cella
# TICKER = "AAPL"     # <-- cambia qui il ticker
# YEARS = 8            # ultimi anni da analizzare

# ==========================================
# UTILITIES
# ==========================================
BASE_URL = "https://discountingcashflows.com/company/{ticker}/{statement}/"

def get_table_discounting(ticker, statement):
    url = BASE_URL.format(ticker=ticker, statement=statement)
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        r = requests.get(url, headers=headers, timeout=20)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta per {statement}: {e}")
        return None

    soup = BeautifulSoup(r.text, "html.parser")
    tables = soup.find_all("table")
    if not tables:
        print(f"Nessuna tabella trovata per {statement} su {url}")
        return None
    for t in tables:
        try:
            df = pd.read_html(io.StringIO(str(t)))[0]
            if df.shape[1] >= 2:
                return df
        except:
            continue
    return None

def extract_row(df, keywords):
    if df is None:
        return None, None  # Return None for both values and periods
    periods = df.columns.tolist()[1:]
    for kw in keywords:
        mask = df.iloc[:, 0].astype(str).str.contains(kw, case=False, na=False)
        if mask.any():
            row = df.loc[mask].iloc[0]
            vals = row.iloc[1:].astype(str).replace("-", np.nan).replace("", np.nan)
            cleaned_vals = []
            cleaned_periods = []
            for i, v in enumerate(vals):
                if pd.isna(v):
                    cleaned_vals.append(np.nan)
                    cleaned_periods.append(periods[i]) # Keep period even if value is NaN
                else:
                    try:
                        s = str(v).replace(",", "").replace("(", "-").replace(")", "").strip()
                        cleaned_vals.append(float(s))
                        cleaned_periods.append(periods[i])
                    except:
                        cleaned_vals.append(np.nan)
                        cleaned_periods.append(periods[i]) # Keep period even if value is NaN
            return cleaned_vals, cleaned_periods
    return None, None # Return None for both values and periods

def safe_ratio(numerator, denominator):
    if denominator is None or np.isnan(denominator) or denominator == 0:
        return np.nan
    if numerator is None or np.isnan(numerator):
        return 0 # Treat missing numerator as 0 for ratio calculation
    # Ensure both are numerical before division
    if not isinstance(numerator, (int, float, np.number)) or not isinstance(denominator, (int, float, np.number)):
         return np.nan # Or raise an error, depending on desired behavior for non-numeric inputs
    return numerator / denominator

def get_latest_fiscal_data(values, periods):
    """
    Given a list of values and periods (most recent first),
    returns the value and period for the latest complete fiscal year
    and the previous fiscal year. Skips LTM data.
    """
    if values is None or periods is None or len(values) < 2 or len(values) != len(periods):
        return None, None, None, None # Need at least 2 values and matching periods

    fiscal_data = [(v, p) for v, p in zip(values, periods) if 'LTM' not in str(p)]

    if len(fiscal_data) < 2:
        return None, None, None, None # Need at least 2 fiscal years

    # fiscal_data is already sorted most recent first from extract_row
    latest_fiscal_value, latest_fiscal_period = fiscal_data[0]
    previous_fiscal_value, previous_fiscal_period = fiscal_data[1]

    return latest_fiscal_value, latest_fiscal_period, previous_fiscal_value, previous_fiscal_period

def get_last_two_periods_data(values, periods):
    """
    Given a list of values and periods (most recent first),
    returns the value and period for the last two available periods,
    including LTM if present.
    """
    if values is None or periods is None or len(values) < 2 or len(values) != len(periods):
        return None, None, None, None # Need at least 2 values and matching periods

    # values and periods are already sorted most recent first from extract_row
    latest_value, latest_period = values[0], periods[0]
    previous_value, previous_period = values[1], periods[1]

    return latest_value, latest_period, previous_value, previous_period

def extract_market_cap_from_overview(ticker):
    """
    Attempts to extract Market Cap from the overview page and infer its scale.
    Returns market cap value and scale factor (e.g., 1e6 for millions, 1e9 for billions).
    """
    url = BASE_URL.format(ticker=ticker, statement="overview")
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        r = requests.get(url, headers=headers, timeout=20)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta per overview: {e}")
        return None, None

    soup = BeautifulSoup(r.text, "html.parser")
    # Use the user's provided logic to find Market Cap text and the next element
    market_cap_text_element = soup.find(string=lambda text: text and "Market Cap" in text)

    if market_cap_text_element:
        # The value and scale are likely in the next sibling element
        value_element = market_cap_text_element.find_next()
        if value_element:
            raw_value_text = value_element.text.strip()
            print(f"Raw Market Cap text found: '{raw_value_text}'") # Debug print

            # Improved logic to parse the value and scale
            import re
            # Regex to find a number (including commas and a potential decimal) followed by optional scale indicator
            match = re.search(r'([\d,]+\.?\d*)\s*(Bil|Mil|Thou|billion|million|thousand)?', raw_value_text, re.IGNORECASE)

            if match:
                value_str = match.group(1).replace(',', '') # Remove commas before converting to float
                scale_indicator = match.group(2)

                try:
                    value = float(value_str)
                    scale_factor = 1.0 # Default scale (base units)

                    if scale_indicator:
                         if scale_indicator.lower() in ['bil', 'billion']:
                             scale_factor = 1e9
                             print("Market Cap scale inferred from overview: Billions")
                         elif scale_indicator.lower() in ['mil', 'million']:
                             scale_factor = 1e6
                             print("Market Cap scale inferred from overview: Millions")
                         elif scale_indicator.lower() in ['thou', 'thousand']:
                             scale_factor = 1e3
                             print("Market Cap scale inferred from overview: Thousands")
                         else:
                              # If indicator is present but not recognized, assume millions as a fallback
                              print(f"Unrecognized scale indicator '{scale_indicator}' found in overview. Assuming millions.")
                              scale_factor = 1e6
                    else:
                         # If no scale indicator is found, try to infer from magnitude (less reliable)
                         # Based on user feedback for this site, assume billions for large companies if no indicator
                         if value > 1e9: # If the raw value is very large, it's likely in base units and needs scaling
                              scale_factor = 1e9 # Assume financial data is in billions, so scale Market Cap to billions
                              print("Market Cap scale inferred from magnitude (assuming billions).")
                         elif value > 1e6:
                              scale_factor = 1e6 # Assume financial data is in millions
                              print("Market Cap scale inferred from magnitude (assuming millions).")
                         else:
                              scale_factor = 1.0
                              print("Market Cap scale inferred from magnitude (assuming base units).")

                    return value, scale_factor

                except ValueError:
                    print(f"Could not parse Market Cap value from overview text: {value_str}")
                    return None, None
            else:
                print(f"Market Cap value and scale indicator not found in expected format in overview text: '{raw_value_text}'")
                return None, None
        else:
            print("Market Cap value element not found after 'Market Cap' text.")
            return None, None

    print("'Market Cap' text not found on overview page.")
    return None, None

# ==========================================
# CALCOLO Z-SCORE, M-SCORE, F-SCORE
# ==========================================
def calculate_scores(ticker, years):
    # Initialize scores and interpretations to default values
    z_score = np.nan
    z_interp = "🔴 Dati insufficienti"
    m_score = np.nan
    m_interp = "🔴 Dati insufficienti"
    f_points = np.nan # Initialize as NaN, will be calculated later if data is sufficient
    f_interp = "🔴 Dati insufficienti"
    f_details = [] # Start with empty details, will be populated during calculation
    overall = "🔴 Dati incompleti per una valutazione affidabile"

    # -------------------------------
    # ESTRAZIONE
    # -------------------------------
    is_df = get_table_discounting(ticker, "income-statement")
    bs_df = get_table_discounting(ticker, "balance-sheet-statement")
    cf_df = get_table_discounting(ticker, "cash-flow-statement")

    # Extract full data and periods (most recent first) - These should be extracted regardless of initial checks
    revenue_full, rev_periods_full = extract_row(is_df, ["Revenue", "Sales"])
    ni_full, ni_periods_full = extract_row(is_df, ["Net Income"])
    gp_full, gp_periods_full = extract_row(is_df, ["Gross Profit"])
    sga_full, sga_periods_full = extract_row(is_df, ["Selling", "General"])
    dep_full, dep_periods_full = extract_row(is_df, ["Depreciation"])
    receiv_full, receiv_periods_full = extract_row(bs_df, ["Receivable"])
    ta_full, ta_periods_full = extract_row(bs_df, ["Total Assets"])
    ca_full, ca_periods_full = extract_row(bs_df, ["Current Assets"])
    ppe_full, ppe_periods_full = extract_row(bs_df, ["Property, Plant", "PPE"])
    td_full, td_periods_full = extract_row(bs_df, ["Total Liabilities"]) # Corrected keyword to be more specific
    cl_full, cl_periods_full = extract_row(bs_df, ["Current Liabilities"]) # Explicitly extract Current Liabilities
    op_income_full, op_income_periods_full = extract_row(is_df, ["Operating Income"])
    cfo_full, cfo_periods_full = extract_row(cf_df, ["Cash Flow from Operations", "Operating Cash Flow"])
    equity_full, equity_periods_full = extract_row(bs_df, ["Total Equity", "Total shareholders' equity", "Total stockholders' equity"])
    fcf_full, fcf_periods_full = extract_row(cf_df, ["Free Cash Flow", "Free cash flow", "FreeCashFlow", "Operating Cash Flow"])

    # Extract specific data for Piotroski points 5 and 7 (as per user request)
    # Point 5: Long-Term Debt and Capital Lease Obligations
    ltd_full, ltd_periods_full = extract_row(bs_df, ["Long-Term Debt", "Long Term Debt"]) # Keywords for Long-Term Debt
    clo_full, clo_periods_full = extract_row(bs_df, ["Capital Lease Obligations", "Capital Lease Liability"]) # Keywords for Capital Lease Obligations
    # Point 7: Shares Outstanding - Extract from Income Statement based on user feedback
    so_full, so_periods_full = extract_row(is_df, ["Diluted Weighted Average Shares Outstanding", "Weighted Average Shares Outstanding", "Shares Outstanding", "Shares", "Common Stock Shares Outstanding", "Weighted Average Shares", "Common Stock Shares"])

    # --- Removed strict initial data check ---
    # Instead, rely on individual index calculations to handle missing data.

    # -------------------------------
    # Get latest available data (first element in the lists) for Z & M Scores (as per user request)
    # -------------------------------
    # Use the absolute latest data available (could be LTM)
    latest_revenue = revenue_full[0] if revenue_full and len(revenue_full) > 0 else np.nan
    latest_ni = ni_full[0] if ni_full and len(ni_full) > 0 else np.nan
    latest_gp = gp_full[0] if gp_full and len(gp_full) > 0 else np.nan
    latest_sga = sga_full[0] if sga_full and len(sga_full) > 0 else np.nan
    latest_dep = dep_full[0] if dep_full and len(dep_full) > 0 else np.nan
    latest_receiv = receiv_full[0] if receiv_full and len(receiv_full) > 0 else np.nan
    latest_ta = ta_full[0] if ta_full and len(ta_full) > 0 else np.nan
    latest_ca = ca_full[0] if ca_full and len(ca_full) > 0 else np.nan
    latest_ppe = ppe_full[0] if ppe_full and len(ppe_full) > 0 else np.nan
    latest_td = td_full[0] if td_full and len(td_full) > 0 else np.nan
    latest_cl = cl_full[0] if cl_full and len(cl_full) > 0 else np.nan
    latest_op_income = op_income_full[0] if op_income_full and len(op_income_full) > 0 else np.nan
    latest_cfo = cfo_full[0] if cfo_full and len(cfo_full) > 0 else np.nan
    latest_equity = equity_full[0] if equity_full and len(equity_full) > 0 else np.nan
    latest_fcf = fcf_full[0] if fcf_full and len(fcf_full) > 0 else np.nan

    # -------------------------------
    # Get data for the last two AVAILABLE periods for Beneish M-Score
    # -------------------------------
    # Use the last two available periods (could include LTM) for M-Score ratios
    latest_receiv_m, latest_period_receiv_m, previous_receiv_m, previous_period_receiv_m = get_last_two_periods_data(receiv_full, receiv_periods_full)
    latest_revenue_m, latest_period_rev_m, previous_revenue_m, previous_period_rev_m = get_last_two_periods_data(revenue_full, rev_periods_full)
    latest_gp_m, latest_period_gp_m, previous_gp_m, previous_period_gp_m = get_last_two_periods_data(gp_full, gp_periods_full)
    latest_ca_m, latest_period_ca_m, previous_ca_m, previous_period_ca_m = get_last_two_periods_data(ca_full, ca_periods_full)
    latest_ppe_m, latest_period_ppe_m, previous_ppe_m, previous_period_ppe_m = get_last_two_periods_data(ppe_full, ppe_periods_full) # Corrected variable name previous_ppe_m
    latest_ta_m, latest_period_ta_m, previous_ta_m, previous_period_ta_m = get_last_two_periods_data(ta_full, ta_periods_full)
    latest_td_m, latest_period_td_m, previous_td_m, previous_period_td_m = get_last_two_periods_data(td_full, td_periods_full)
    latest_equity_m, latest_period_equity_m, previous_equity_m, previous_period_equity_m = get_last_two_periods_data(equity_full, equity_periods_full)
    latest_dep_m, latest_period_dep_m, previous_dep_m, previous_period_dep_m = get_last_two_periods_data(dep_full, dep_periods_full)
    latest_sga_m, latest_period_sga_m, previous_sga_m, previous_period_sga_m = get_last_two_periods_data(sga_full, sga_periods_full)

    # -------------------------------
    # Get data for the last two AVAILABLE periods for Piotroski F-Score (as per user request)
    # -------------------------------
    # Now Piotroski will also use the last two available periods (could include LTM)
    # Get data for all required Piotroski metrics for the last two periods
    latest_ni_f, latest_period_ni_f, previous_ni_f, previous_period_ni_f = get_last_two_periods_data(ni_full, ni_periods_full)
    latest_cfo_f, latest_period_cfo_f, previous_cfo_f, previous_period_cfo_f = get_last_two_periods_data(cfo_full, cfo_periods_full)
    latest_ta_f, latest_period_ta_f, previous_ta_f, previous_period_ta_f = get_last_two_periods_data(ta_full, ta_periods_full)
    latest_cl_f, latest_period_cl_f, previous_cl_f, previous_period_cl_f = get_last_two_periods_data(cl_full, cl_periods_full)
    latest_equity_f, latest_period_equity_f, previous_equity_f, previous_period_equity_f = get_last_two_periods_data(equity_full, equity_periods_full)
    latest_revenue_f, latest_period_rev_f, previous_revenue_f, previous_period_rev_f = get_last_two_periods_data(revenue_full, rev_periods_full)
    latest_gp_f, latest_period_gp_f, previous_gp_f, previous_period_gp_f = get_last_two_periods_data(gp_full, gp_periods_full)
    latest_td_f, latest_period_td_f, previous_td_f, previous_period_td_f = get_last_two_periods_data(td_full, td_periods_full)
    latest_ca_f, latest_period_ca_f, previous_ca_f, previous_period_ca_f = get_last_two_periods_data(ca_full, ca_periods_full)
    latest_ltd_f, latest_period_ltd_f, previous_ltd_f, previous_period_ltd_f = get_last_two_periods_data(ltd_full, ltd_periods_full)
    latest_clo_f, latest_period_clo_f, previous_clo_f, previous_period_clo_f = get_last_two_periods_data(clo_full, clo_periods_full)
    latest_so_f, latest_period_so_f, previous_so_f, previous_period_so_f = get_last_two_periods_data(so_full, so_periods_full) # Use so_full extracted from IS_df

    # -------------------------------
    # CALCOLO VARIABILI PER M-SCORE (using last two available periods)
    # -------------------------------
    # Use last two available periods for year-over-year ratios in M-Score
    DSRI = safe_ratio(safe_ratio(latest_receiv_m, latest_revenue_m),
                       safe_ratio(previous_receiv_m, previous_revenue_m))

    GMI = safe_ratio(safe_ratio(previous_gp_m, previous_revenue_m),
                       safe_ratio(latest_gp_m, latest_revenue_m))

    # Ensure inputs to safe_ratio are numerical before the addition
    aq_latest_m = 1 - safe_ratio((latest_ca_m if isinstance(latest_ca_m, (int, float, np.number)) else np.nan) + (latest_ppe_m if isinstance(latest_ppe_m, (int, float, np.number)) else np.nan), latest_ta_m)
    aq_previous_m = 1 - safe_ratio((previous_ca_m if isinstance(previous_ca_m, (int, float, np.number)) else np.nan) + (previous_ppe_m if isinstance(previous_ppe_m, (int, float, np.number)) else np.nan), previous_ta_m)

    AQI = safe_ratio(aq_latest_m, aq_previous_m)

    SGI = safe_ratio(latest_revenue_m, previous_revenue_m)

    # TATA uses latest available data (could be LTM), no previous period needed for the ratio itself
    # Using latest_ni, latest_cfo, latest_ta which are from the absolute latest period
    TATA = safe_ratio((latest_ni if isinstance(latest_ni, (int, float, np.number)) else np.nan) - (latest_cfo if isinstance(latest_cfo, (int, float, np.number)) else np.nan), latest_ta)

    LVGI = safe_ratio(safe_ratio(latest_td_m, latest_equity_m),
                       safe_ratio(previous_td_m, previous_equity_m))

    # DEPI (Depreciation Index) - Ratio of depreciation rate in current year to previous year
    # Depreciation rate = Depreciation / (PPE + Depreciation)
    # Ensure inputs to safe_ratio are numerical before the addition
    dep_rate_latest_m = safe_ratio(latest_dep_m, (latest_ppe_m if isinstance(latest_ppe_m, (int, float, np.number)) else np.nan) + (latest_dep_m if isinstance(latest_dep_m, (int, float, np.number)) else np.nan))
    dep_rate_previous_m = safe_ratio(previous_dep_m, (previous_ppe_m if isinstance(previous_ppe_m, (int, float, np.number)) else np.nan) + (previous_dep_m if isinstance(previous_dep_m, (int, float, np.number)) else np.nan))
    DEPI = safe_ratio(dep_rate_previous_m, dep_rate_latest_m) # Note: Formula in user's example seems to use previous/current

    # SGAI (Sales, General, and Administrative Expenses Index) - Ratio of SGA to Sales in current year to previous year
    # SGA to Sales ratio = SGA / Revenue
    sga_sales_ratio_latest_m = safe_ratio(latest_sga_m, latest_revenue_m)
    sga_sales_ratio_previous_m = safe_ratio(previous_sga_m, previous_revenue_m)
    SGAI = safe_ratio(sga_sales_ratio_latest_m, sga_sales_ratio_previous_m)

    # === BENEISH M-SCORE ===
    # Calculate M-Score after calculating its components
    # Use the 8-variable formula based on user's provided calculation
    # M = -4.84 + 0.92 * DSRI + 0.528 * GMI + 0.404 * AQI + 0.892 * SGI + 0.115 * DEPI - 0.172 * SGAI + 4.679 * TATA - 0.327 * LVGI
    m_score_components = {
        'DSRI': DSRI,
        'GMI': GMI,
        'AQI': AQI,
        'SGI': SGI,
        'TATA': TATA,
        'LVGI': LVGI,
        'DEPI': DEPI,
        'SGAI': SGAI
    }

    # Check if all required variables for the 8-variable M-Score are available (not NaN)
    if any(np.isnan(list(m_score_components.values()))):
        m_score = np.nan
    else:
         # 8-variable Beneish M-Score formula
         m_score = (-4.84
                    + 0.92 * m_score_components['DSRI']
                    + 0.528 * m_score_components['GMI']
                    + 0.404 * m_score_components['AQI']
                    + 0.892 * m_score_components['SGI']
                    + 0.115 * m_score_components['DEPI']
                    - 0.172 * m_score_components['SGAI']
                    + 4.679 * m_score_components['TATA']
                    - 0.327 * m_score_components['LVGI'])

    if np.isnan(m_score):
        m_interp = "🔴 Dati insufficienti"
    # Note: The typical cutoff for the 8-variable model is often -1.78
    elif m_score < -1.78:
        m_interp = "💚 Bassa probabilità di manipolazione (secondo il modello a 8 variabili)"
    else:
        m_interp = "🔴 Possibile manipolazione contabile (secondo il modello a 8 variabili)"

    # === ALTMAN Z-SCORE ===
    # Use latest available data for Altman Z-Score components (as per user request)
    try:
        total_assets_altman = latest_ta
        total_liabilities_altman = latest_td
        current_assets_altman = latest_ca
        current_liabilities_altman = latest_cl
        equity_altman = latest_equity
        revenue_altman = latest_revenue
        ni_altman = latest_ni
        dep_altman = latest_dep
        op_income_altman = op_income_full[0] if op_income_full and len(op_income_full) > 0 else np.nan # Use latest Op Income

        # Handle cases where data might have NaNs or be insufficient for base Z-score components
        if np.isnan(total_assets_altman) or total_assets_altman == 0 or \
           np.isnan(current_assets_altman) or np.isnan(current_liabilities_altman) or \
           np.isnan(revenue_altman) or np.isnan(ni_altman): # Added ni_altman check for RE and EBIT
             z_score = np.nan
             z_interp = "🔴 Dati insufficienti"
             # Store error message for later printing
             altman_error_message = "Altman Z-Score Data (Latest Available): Insufficient base data for calculation."
        else:
            # Use Operating Income if available, otherwise approximate EBIT (using latest available data)
            if op_income_altman is not None and not np.isnan(op_income_altman):
                ebit_altman = op_income_altman
            elif ni_altman is not None and not np.isnan(ni_altman) and dep_altman is not None and not np.isnan(dep_altman):
                 ebit_altman = ni_altman + dep_altman
            else:
                 ebit_altman = np.nan # EBIT might be NaN if both Op Income and NI+Dep are missing

            # Get Market Cap - Prioritize overview page
            market_cap_altman_value = np.nan # Initialize as NaN
            market_cap_scale_factor = np.nan # Initialize scale factor

            overview_mc_value, overview_scale = extract_market_cap_from_overview(ticker)

            if overview_mc_value is not None and overview_scale is not None:
                 market_cap_altman_value = overview_mc_value
                 market_cap_scale_factor = overview_scale # Use the scale inferred from overview
                 print(f"Market Cap value from overview: {market_cap_altman_value}, Scale Factor: {market_cap_scale_factor}")

                 # Calculate components using the latest available data point
                 wc_ta = safe_ratio((current_assets_altman if isinstance(current_assets_altman, (int, float, np.number)) else np.nan) - (current_liabilities_altman if isinstance(current_liabilities_altman, (int, float, np.number)) else np.nan), total_assets_altman)
                 # For Retained Earnings approximation, sum Net Income over the specified years (most recent first)
                 # Use ni_full which contains all extracted Net Income values
                 retained_earnings_altman_approx = np.nansum(ni_full[:years]) # Sum over the last 'years' extracted
                 re_ta = safe_ratio(retained_earnings_altman_approx, total_assets_altman)

                 ebit_ta = safe_ratio(ebit_altman, total_assets_altman)

                 # Use Market Value of Equity (Market Cap) and Total Liabilities (latest available)
                 # Ensure both are non-NaN before ratio calculation.
                 # Scale Market Cap to the same scale as Total Liabilities (assuming millions based on previous output).
                 assumed_financial_scale = 1e6 # Assuming financial data (like Total Liabilities) is in millions
                 if not np.isnan(market_cap_altman_value) and not np.isnan(total_liabilities_altman) and total_liabilities_altman != 0 and not np.isnan(market_cap_scale_factor):
                      # Convert Market Cap value from its extracted scale to the assumed financial data scale (millions)
                      scaled_market_cap = market_cap_altman_value * (market_cap_scale_factor / assumed_financial_scale)
                      mve_tl = safe_ratio(scaled_market_cap, total_liabilities_altman) # Total Liabilities assumed to be in millions
                 else:
                      mve_tl = np.nan # X4 will be NaN if Market Cap or its scale is missing

                 s_ta = safe_ratio(revenue_altman, total_assets_altman)

                 # Calculate Z-Score only if all components are available
                 if not any(np.isnan([wc_ta, re_ta, ebit_ta, mve_tl, s_ta])):
                      z_score = 1.2*wc_ta + 1.4*re_ta + 3.3*ebit_ta + 0.6*mve_tl + 1.0*s_ta
                      if z_score > 2.99: z_interp = "💚 Solida (basso rischio di fallimento)"
                      elif z_score >= 1.81: z_interp = "🟡 Zona grigia (moderato rischio)"
                      else: z_interp = "🔴 Rischio elevato di insolvenza"
                 else:
                      z_score = np.nan
                      z_interp = "🔴 Dati insufficienti"

                 # Store Altman Z-Score Components Details for later printing
                 altman_component_prints = [
                     f"WC/TA (X1): {wc_ta:.4f}",
                     f"RE/TA (X2): {re_ta:.4f}",
                     f"EBIT/TA (X3): {ebit_ta:.4f}",
                     f"MVE/TL (X4): {mve_tl:.4f}" if not np.isnan(mve_tl) else "MVE/TL (X4): N/A (Market Cap Missing/Scale Unclear)",
                     f"S/TA (X5): {s_ta:.4f}"
                 ]
                 altman_error_message = None

            else:
                 print("⚠️ Market Cap not found on overview page with clear scale. Cannot calculate X4 accurately.")
                 # If overview fails, set Z-score related variables to NaN/default
                 mve_tl = np.nan
                 z_score = np.nan
                 z_interp = "🔴 Dati insufficienti"
                 # Store Altman Z-Score Components Details for later printing (with N/A for X4)
                 wc_ta = safe_ratio((current_assets_altman if isinstance(current_assets_altman, (int, float, np.number)) else np.nan) - (current_liabilities_altman if isinstance(current_liabilities_altman, (int, float, np.number)) else np.nan), total_assets_altman)
                 retained_earnings_altman_approx = np.nansum(ni_full[:years]) # Sum over the last 'years' extracted
                 re_ta = safe_ratio(retained_earnings_altman_approx, total_assets_altman)
                 ebit_ta = safe_ratio(ebit_altman, total_assets_altman)
                 s_ta = safe_ratio(revenue_altman, total_assets_altman)
                 altman_component_prints = [
                     f"WC/TA (X1): {wc_ta:.4f}",
                     f"RE/TA (X2): {re_ta:.4f}",
                     f"EBIT/TA (X3): {ebit_ta:.4f}",
                     f"MVE/TL (X4): N/A (Market Cap Missing/Scale Unclear)",
                     f"S/TA (X5): {s_ta:.4f}"
                 ]
                 altman_error_message = "Altman Z-Score Data (Latest Available): Insufficient Market Cap data with reliable scale for X4 calculation."

    except Exception as e:
        print(f"Errore calcolo Z-Score: {e}")
        z_score = np.nan
        z_interp = "🔴 Dati insufficienti"
        altman_component_prints = [] # Clear components on general error
        altman_error_message = f"Errore calcolo Z-Score: {e}"

    # === PIOTROSKI F-SCORE (using last two available periods) ===
    f_points = 0 # Initialize points to 0, will add points based on available data
    f_details = [] # Reset f_details

    try:
        # Profitability criteria (4 points)
        # 1. Positive Net Income (Latest Period)
        if latest_ni_f is not None and not np.isnan(latest_ni_f) and latest_ni_f > 0:
            f_points += 1
            f_details.append(f"1. Net Income ({latest_ni_f:,.0f}) > 0 (+1 point) [{latest_period_ni_f}]") # Added period
        else:
            f_details.append(f"1. Net Income ({latest_ni_f if latest_ni_f is not None and not np.isnan(latest_ni_f) else 'N/A'} @{latest_period_ni_f}) <= 0 or data missing (+0 points)") # Added period

        # 2. Positive Cash Flow from Operations (Latest Period)
        if latest_cfo_f is not None and not np.isnan(latest_cfo_f) and latest_cfo_f > 0:
            f_points += 1
            f_details.append(f"2. CFO ({latest_cfo_f:,.0f}) > 0 (+1 point) [{latest_period_cfo_f}]") # Added period
        else:
             f_details.append(f"2. CFO ({latest_cfo_f if latest_cfo_f is not None and not np.isnan(latest_cfo_f) else 'N/A'} @{latest_period_cfo_f}) <= 0 or data missing (+0 points)") # Added period

        # 3. Return on Assets (ROA) improvement (Latest Period vs Previous Period)
        roa_curr = safe_ratio(latest_ni_f, latest_ta_f)
        roa_prev = safe_ratio(previous_ni_f, previous_ta_f)
        if not np.isnan(roa_curr) and not np.isnan(roa_prev) and roa_curr > roa_prev:
             f_points += 1
             f_details.append(f"3. ROA ({roa_curr:.4f} @{latest_period_ni_f}/{latest_period_ta_f}) > Previous ROA ({roa_prev:.4f} @{previous_period_ni_f}/{previous_period_ta_f}) (+1 point)") # Added periods
        else:
             f_details.append(f"3. ROA ({roa_curr:.4f}) <= Previous ROA ({roa_prev:.4f}) or data missing (+0 points) [Latest @{latest_period_ni_f}/{latest_period_ta_f}, Previous @{previous_period_ni_f}/{previous_period_ta_f}]") # Added periods

        # 4. Cash Flow from Operations (CFO) > Net Income (Latest Period)
        if not np.isnan(latest_cfo_f) and not np.isnan(latest_ni_f) and latest_cfo_f > latest_ni_f:
             f_points += 1
             f_details.append(f"4. CFO ({latest_cfo_f:,.0f}) > Net Income ({latest_ni_f:,.0f}) (+1 point) [Both @{latest_period_cfo_f}/{latest_period_ni_f}]") # Added periods
        else:
             f_details.append(f"4. CFO ({latest_cfo_f:,.0f}) <= Net Income ({latest_ni_f:,.0f}) or data missing (+0 points) [Both @{latest_period_cfo_f}/{latest_period_ni_f}]") # Added periods

        # Leverage, Liquidity and Source of Funds criteria (3 points)
        # 5. Lower Long-Term Debt + Capital Lease Obligations in the current year compared to the previous year (Latest Period vs Previous Period)
        # Using sum of LTD and CLO
        latest_total_long_term_debt = (latest_ltd_f if latest_ltd_f is not None and not np.isnan(latest_ltd_f) else 0) + \
                                      (latest_clo_f if latest_clo_f is not None and not np.isnan(latest_clo_f) else 0)
        previous_total_long_term_debt = (previous_ltd_f if previous_ltd_f is not None and not np.isnan(previous_ltd_f) else 0) + \
                                        (previous_clo_f if previous_clo_f is not None and not np.isnan(previous_clo_f) else 0)
        # Check if data for comparison exists for at least one component in both periods
        if (latest_ltd_f is not None and not np.isnan(latest_ltd_f)) or (latest_clo_f is not None and not np.isnan(latest_clo_f)) or \
           (previous_ltd_f is not None and not np.isnan(previous_ltd_f)) or (previous_clo_f is not None and not np.isnan(previous_clo_f)):
             # Only compare if we have data for at least one component in both periods
             if latest_total_long_term_debt < previous_total_long_term_debt:
                  f_points += 1
                  f_details.append(f"5. Long-Term Debt + CLO ({latest_total_long_term_debt:,.0f} @{latest_period_ltd_f}/{latest_period_clo_f}) < Previous ({previous_total_long_term_debt:,.0f} @{previous_period_ltd_f}/{previous_period_clo_f}) (+1 point)") # Added periods
             else:
                  f_details.append(f"5. Long-Term Debt + CLO ({latest_total_long_term_debt:,.0f} @{latest_period_ltd_f}/{latest_period_clo_f}) >= Previous ({previous_total_long_term_debt:,.0f} @{previous_period_ltd_f}/{previous_period_clo_f}) (+0 points)") # Added periods
        else:
             f_details.append(f"5. Long-Term Debt + CLO data insufficient for comparison (+0 points) [Latest @{latest_period_ltd_f}/{latest_period_clo_f}, Previous @{previous_period_ltd_f}/{previous_period_clo_f}]") # Added periods

        # 6. Higher Current Ratio in the current year compared to the previous year (Latest Period vs Previous Period)
        cr_curr = safe_ratio(latest_ca_f, latest_cl_f)
        cr_prev = safe_ratio(previous_ca_f, previous_cl_f)
        if not np.isnan(cr_curr) and not np.isnan(cr_prev) and cr_curr > cr_prev:
             f_points += 1
             f_details.append(f"6. Current Ratio ({cr_curr:.4f} @{latest_period_ca_f}/{latest_period_cl_f}) > Previous Current Ratio ({cr_prev:.4f} @{previous_period_ca_f}/{previous_period_cl_f}) (+1 point)") # Added periods
        else:
             f_details.append(f"6. Current Ratio ({cr_curr:.4f}) <= Previous Current Ratio ({cr_prev:.4f}) or data missing (+0 points) [Latest @{latest_period_ca_f}/{latest_period_cl_f}, Previous @{previous_period_ca_f}/{previous_period_cl_f}]") # Added periods

        # 7. No new shares issued in the last year (signified by stable or decreasing shares outstanding) (Latest Period vs Previous Period)
        # Comparing Shares Outstanding directly
        if latest_so_f is not None and not np.isnan(latest_so_f) and previous_so_f is not None and not np.isnan(previous_so_f):
             if latest_so_f <= previous_so_f: # Score 1 if shares outstanding is less than or equal to previous
                  f_points += 1
                  f_details.append(f"7. Shares Outstanding ({latest_so_f:,.0f} @{latest_period_so_f}) <= Previous ({previous_so_f:,.0f} @{previous_period_so_f}) (+1 point)") # Added periods
             else:
                  f_details.append(f"7. Shares Outstanding ({latest_so_f:,.0f} @{latest_period_so_f}) > Previous ({previous_so_f:,.0f} @{previous_period_so_f}) (+0 points)") # Added periods
        else:
             f_details.append(f"7. Shares Outstanding data insufficient for comparison (+0 points) [Latest @{latest_period_so_f}, Previous @{previous_period_so_f}]") # Added periods

        # Operating Efficiency criteria (2 points) - Changed to 2 points as point 4 moved to Profitability
        # 8. Higher Gross Margin in the current year compared to the previous year (Latest Period vs Previous Period)
        gm_curr = safe_ratio(latest_gp_f, latest_revenue_f)
        gm_prev = safe_ratio(previous_gp_f, previous_revenue_f)
        if not np.isnan(gm_curr) and not np.isnan(gm_prev) and gm_curr > gm_prev:
             f_points += 1
             f_details.append(f"8. Gross Margin ({gm_curr:.4f} @{latest_period_gp_f}/{latest_period_rev_f}) > Previous Gross Margin ({gm_prev:.4f} @{previous_period_gp_f}/{previous_period_rev_f}) (+1 point)") # Added periods
        else:
             f_details.append(f"8. Gross Margin ({gm_curr:.4f}) <= Previous Gross Margin ({gm_prev:.4f}) or data missing (+0 points) [Latest @{latest_period_gp_f}/{latest_period_rev_f}, Previous @{previous_period_gp_f}/{previous_period_rev_f}]") # Added periods

        # 9. Higher Asset Turnover Ratio in the current year compared to the previous year (Latest Period vs Previous Period)
        # Note: Asset Turnover = Revenue / Total Assets
        at_curr = safe_ratio(latest_revenue_f, latest_ta_f)
        at_prev = safe_ratio(previous_revenue_f, previous_ta_f)
        if not np.isnan(at_curr) and not np.isnan(at_prev) and at_curr > at_prev:
             f_points += 1
             f_details.append(f"9. Asset Turnover ({at_curr:.4f} @{latest_period_rev_f}/{latest_period_ta_f}) > Previous Asset Turnover ({at_prev:.4f} @{previous_period_rev_f}/{previous_period_ta_f}) (+1 point)") # Added periods
        else:
             f_details.append(f"9. Asset Turnover ({at_curr:.4f}) <= Previous Asset Turnover ({at_prev:.4f}) or data missing (+0 points) [Latest @{latest_period_rev_f}/{latest_period_ta_f}, Previous @{previous_period_rev_f}/{previous_period_ta_f}]") # Added periods

        # Set interpretation based on calculated points.
        # Check if at least some key data was available to attempt calculation
        core_data_for_fscore_available = (latest_ni_f is not None and not np.isnan(latest_ni_f) and previous_ni_f is not None and not np.isnan(previous_ni_f)) or \
                                       (latest_cfo_f is not None and not np.isnan(latest_cfo_f) and previous_cfo_f is not None and not np.isnan(previous_cfo_f)) or \
                                       (latest_ta_f is not None and not np.isnan(latest_ta_f) and previous_ta_f is not None and not np.isnan(previous_ta_f)) or \
                                       (latest_cl_f is not None and not np.isnan(latest_cl_f) and previous_cl_f is not None and not np.isnan(previous_cl_f)) or \
                                       (latest_equity_f is not None and not np.isnan(latest_equity_f) and previous_equity_f is not None and not np.isnan(previous_equity_f)) or \
                                       (latest_revenue_f is not None and not np.isnan(latest_revenue_f) and previous_revenue_f is not None and not np.isnan(previous_revenue_f)) or \
                                       (latest_gp_f is not None and not np.isnan(latest_gp_f) and previous_gp_f is not None and not np.isnan(previous_gp_f)) or \
                                       (latest_td_f is not None and not np.isnan(latest_td_f) and previous_td_f is not None and not np.isnan(previous_td_f)) or \
                                       (latest_ca_f is not None and not np.isnan(latest_ca_f) and previous_ca_f is not None and not np.isnan(previous_ca_f)) or \
                                       (latest_ltd_f is not None and not np.isnan(latest_ltd_f) and previous_ltd_f is not None and not np.isnan(previous_ltd_f)) or \
                                       (latest_clo_f is not None and not np.isnan(latest_clo_f) and previous_clo_f is not None and not np.isnan(previous_clo_f)) or \
                                       (latest_so_f is not None and not np.isnan(latest_so_f) and previous_so_f is not None and not np.isnan(previous_so_f)) # Check all required metrics for last two periods

        if not core_data_for_fscore_available:
             f_interp = "🔴 Dati insufficienti per calcolare Punteggio F-Score" # Specific message if fundamental data is missing
             f_points = np.nan # Set to NaN if fundamental data is missing
             # f_details already contains specific missing data points from individual checks
        # elif f_details and all("data insufficient" in detail for detail in f_details):
             # f_interp = "🔴 Dati insufficienti per calcolare Punteggio F-Score" # More specific message if all points missed due to data
             # f_points = np.nan # Set to NaN if all points missed due to data
        elif f_points >= 8:
            f_interp = "💚 Molto solida (ottima efficienza)"
        elif f_points >= 5:
            f_interp = "🟡 Solida ma migliorabile"
        else:
            f_interp = "🔴 Debole (rischi elevati)"

    except Exception as e:
         print(f"Errore calcolo F-Score: {e}")
         f_points = np.nan # Set to NaN on general error during calculation
         f_interp = "🔴 Dati insufficienti"
         f_details = [f"Error during F-Score calculation: {e}"] # Update details with error

    # --- Debugging: Print data fetched for Piotroski ---
    print("\n--- Debugging Piotroski Data Extraction (Last Two Periods) ---")
    print(f"Net Income (NI): Latest={latest_ni_f}, Period={latest_period_ni_f} | Previous={previous_ni_f}, Period={previous_period_ni_f}")
    print(f"CFO: Latest={latest_cfo_f}, Period={latest_period_cfo_f} | Previous={previous_cfo_f}, Period={previous_period_cfo_f}")
    print(f"Total Assets (TA): Latest={latest_ta_f}, Period={latest_period_ta_f} | Previous={previous_ta_f}, Period={previous_period_ta_f}")
    print(f"Current Liabilities (CL): Latest={latest_cl_f}, Period={latest_period_cl_f} | Previous={previous_cl_f}, Period={previous_period_cl_f}")
    print(f"Equity: Latest={latest_equity_f}, Period={latest_period_equity_f} | Previous={previous_equity_f}, Period={previous_period_equity_f}")
    print(f"Revenue: Latest={latest_revenue_f}, Period={latest_period_rev_f} | Previous={previous_revenue_f}, Period={previous_period_rev_f}")
    print(f"Gross Profit (GP): Latest={latest_gp_f}, Period={latest_period_gp_f} | Previous={previous_gp_f}, Period={previous_period_gp_f}")
    print(f"Total Liabilities (TD): Latest={latest_td_f}, Period={latest_period_td_f} | Previous={previous_td_f}, Period={previous_period_td_f}")
    print(f"Current Assets (CA): Latest={latest_ca_f}, Period={latest_period_ca_f} | Previous={previous_ca_f}, Period={previous_period_ca_f}")
    print(f"Long-Term Debt (LTD): Latest={latest_ltd_f}, Period={latest_period_ltd_f} | Previous={previous_ltd_f}, Period={previous_period_ltd_f}")
    print(f"Capital Lease Obligations (CLO): Latest={latest_clo_f}, Period={latest_period_clo_f} | Previous={previous_clo_f}, Period={previous_period_clo_f}")
    print(f"Shares Outstanding (SO): Latest={latest_so_f}, Period={latest_period_so_f} | Previous={previous_so_f}, Period={previous_period_so_f}")
    print("----------------------------------------------------------")
    # --- End Debugging Prints ---

    # Print Altman Z-Score details if available
    # Check if Altman components were calculated or an error occurred in that block
    if 'altman_component_prints' in locals() or ('altman_error_message' in locals() and altman_error_message is not None):
        print("\n--- Altman Z-Score Components Details ---")
        if 'altman_error_message' in locals() and altman_error_message:
             print(altman_error_message)
        elif 'altman_component_prints' in locals() and altman_component_prints:
             for comp in altman_component_prints:
                 print(comp)
        else:
            # Should not happen if calculation was attempted, but as a fallback
            print("Altman Z-Score component details not available.")
        print("---------------------------------------------------------")

    # Print Beneish M-Score details if available
    # Check if M-Score components were calculated (even if m_score is NaN)
    if any(comp in m_score_components for comp in ['DSRI', 'GMI', 'AQI', 'SGI', 'TATA', 'LVGI', 'DEPI', 'SGAI']):
        print("\n--- Beneish M-Score Components Details ---")
        # Print details for each component, including underlying values for ratios (using the M-Score specific data)
        print(f"DSRI (Days Sales in Receivables Index): {m_score_components.get('DSRI', np.nan):.4f}")
        print(f"  Latest Period ({latest_period_receiv_m}): Receivables: {latest_receiv_m:,.0f}, Revenue: {latest_revenue_m:,.0f}")
        print(f"  Previous Period ({previous_period_receiv_m}): Receivables: {previous_receiv_m:,.0f}, Revenue: {previous_revenue_m:,.0f}")

        print(f"GMI (Gross Margin Index): {m_score_components.get('GMI', np.nan):.4f}")
        print(f"  Latest Period ({latest_period_gp_m}): Gross Profit: {latest_gp_m:,.0f}, Revenue: {latest_revenue_m:,.0f}")
        print(f"  Previous Period ({previous_period_gp_m}): Gross Profit: {previous_gp_m:,.0f}, Revenue: {previous_revenue_m:,.0f}")

        print(f"AQI (Asset Quality Index): {AQI:.4f}") # Use AQI variable directly
        print(f"  Latest Period ({latest_period_ca_m}): CA: {latest_ca_m:,.0f}, PPE: {latest_ppe_m:,.0f}, TA: {latest_ta_m:,.0f}")
        print(f"  Previous Period ({previous_period_ca_m}): CA: {previous_ca_m:,.0f}, PPE: {previous_ppe_m:,.0f}, TA: {previous_ta_m:,.0f}")

        print(f"SGI (Sales Growth Index): {m_score_components.get('SGI', np.nan):.4f}")
        print(f"  Latest Period ({latest_period_rev_m}): Revenue: {latest_revenue_m:,.0f}")
        print(f"  Previous Period ({previous_period_rev_m}): Revenue: {previous_revenue_m:,.0f}")

        print(f"TATA (Total Accruals to Total Assets): {m_score_components.get('TATA', np.nan):.4f}")
        print(f"  Latest Net Income: {latest_ni:,.0f}, Latest CFO: {latest_cfo:,.0f}, Latest Total Assets: {latest_ta:,.0f}")

        print(f"LVGI (Leverage Index): {m_score_components.get('LVGI', np.nan):.4f}") # Corrected format specifier
        print(f"  Latest Period ({latest_period_td_m}): Total Liabilities: {latest_td_m:,.0f}, Equity: {latest_equity_m:,.0f}")
        print(f"  Previous Period ({previous_period_td_m}): Total Liabilities: {previous_td_m:,.0f}, Equity: {previous_equity_m:,.0f}")

        print(f"DEPI (Depreciation Index): {m_score_components.get('DEPI', np.nan):.4f}")
        print(f"  Latest Period ({latest_period_dep_m}): Depreciation: {latest_dep_m:,.0f}, PPE: {latest_ppe_m:,.0f}")
        print(f"  Previous Period ({previous_period_dep_m}): Depreciation: {previous_dep_m:,.0f}, PPE: {previous_ppe_m:,.0f}")

        print(f"SGAI (SGA Index): {m_score_components.get('SGAI', np.nan):.4f}")
        print(f"  Latest Period ({latest_period_sga_m}): SGA: {latest_sga_m:,.0f}, Revenue: {latest_revenue_m:,.0f}")
        print(f"  Previous Period ({previous_period_sga_m}): SGA: {previous_sga_m:,.0f}, Revenue: {previous_revenue_m:,.0f}")

        print("------------------------------------------")

    # Print Piotroski F-Score details
    print("\nPiotroski F-Score Details:")
    for detail in f_details:
        print(f"- {detail}")

    # Print Summary Scores and Combined Interpretation at the very end
    print(f"\n📊 Analisi di solidità per: {ticker}") # Moved this line here
    print("\n--- Riepilogo Punteggi ---")
    print(f"Altman Z-Score: {z_score:.2f} → {z_interp}")
    print(f"Beneish M-Score: {m_score:.2f} → {m_interp}")
    print(f"Piotroski F-Score: {f_points}/9 → {f_interp}") # f_points might be NaN or a number
    print("-------------------------")

    # === INTERPRETAZIONE COMBINATA ===
    # Check if ALL scores have a valid, non-NaN value before providing a combined interpretation
    if not np.isnan(z_score) and not np.isnan(m_score) and (f_points is not None and not np.isnan(f_points)): # Added check for f_points
         # Use the 8-variable cutoff for M-Score in combined interpretation
         if z_score > 2.99 and m_score < -1.78 and f_points >= 8:
             overall = "💚 Eccellente solidità e bilanci affidabili"
         elif z_score >= 1.81 and m_score < -1.78 and f_points >= 5:
             overall = "🟡 Solida ma da monitorare"
         else:
             overall = "🔴 Rischio elevato o possibili manipolazioni"
    else:
         overall = "🔴 Dati incompleti per una valutazione affidabile" # Default if any score is NaN

    print(f"\n🧠 Interpretazione combinata: {overall}")

# ==========================================
# ESECUZIONE COMPLETA
# ==========================================
# Make sure TICKER and YEARS are defined (e.g., from the first cell)
if 'TICKER' in globals() and 'YEARS' in globals():
    calculate_scores(TICKER, YEARS)
else:
    print("⚠️ TICKER and/o r YEARS variables are not defined. Please run the first cell.")